<a href="https://colab.research.google.com/github/SAIKUMAR918/Deep_Learning_Projects/blob/main/LinearRegression%26ANN_HousingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Required Libraries:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
# Library to build Neural Networks:
import tensorflow as tf
from tensorflow import keras

In [3]:
# Fetching the data from sklearn:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

In [4]:
housing.data

array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

In [5]:
# converting the data into a data frame:
data=pd.DataFrame(housing.data)

In [7]:
# To display first five records:
data.head()

,0,1,2,3,4,5,6,7
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [8]:
# Assigning names to columns:
data.columns = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


### Data Set Characteristics:

##### Number of Instances:

20640

##### Number of Attributes:

8 numeric, predictive attributes and the target

##### Attribute Information:

* MedInc median income in block
* HouseAge median house age in block
* AveRooms average number of rooms
* AveBedrms average number of bedrooms
* Population block population
* AveOccup average house occupancy
* Latitude house block latitude
* Longitude house block longitude

#### Target

The target variable is the median house value in units of 100,000 for California districts.

In [9]:
print(housing.feature_names)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [10]:
housing.target

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [11]:
# Splitting the data:
from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [12]:
y_train_full

array([2.285, 2.799, 1.83 , ..., 2.221, 2.835, 3.25 ])

In [13]:
X_train.shape

(11610, 8)

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [15]:
# print the splitting of data:
print('X train size',len(X_train))
print('X test size',len(X_test))
print('y train size',len(y_train))
print('y test size',len(y_test))

X train size 11610
X test size 5160
y train size 11610
y test size 5160


# MODEL BUILDING

## Linear Regression Model Using Stats

In [16]:
import statsmodels.api as sm
x_reg=sm.add_constant(X_train)
x_reg=sm.add_constant(X_test)

In [17]:
lin_model=sm.OLS(y_train,X_train).fit()
print(lin_model.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.144
Model:                            OLS   Adj. R-squared (uncentered):              0.143
Method:                 Least Squares   F-statistic:                              243.4
Date:                Wed, 13 Sep 2023   Prob (F-statistic):                        0.00
Time:                        05:27:18   Log-Likelihood:                         -25652.
No. Observations:               11610   AIC:                                  5.132e+04
Df Residuals:                   11602   BIC:                                  5.138e+04
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [18]:
from sklearn import metrics
import numpy as np
scores = pd.DataFrame(columns=['Model','MAE_Train','MSE_Train','RMSE_Train','MAPE_Train','MAE_Test','MSE_Test','RMSE_Test','MAPE_Test'])

def get_metrics(train_act,train_pred,test_act,test_pred,model_description,dataframe):
    MAE_Train = metrics.mean_absolute_error(train_act,train_pred)
    MSE_Train = metrics.mean_squared_error(train_act,train_pred)
    RMSE_Train = np.sqrt(metrics.mean_squared_error(train_act,train_pred))
    MAPE_Train = metrics.mean_absolute_percentage_error(train_act,train_pred)
    MAE_Test = metrics.mean_absolute_error(test_act,test_pred)
    MSE_Test = metrics.mean_squared_error(test_act,test_pred)
    RMSE_Test = np.sqrt(metrics.mean_squared_error(test_act,test_pred))
    MAPE_Test = metrics.mean_absolute_percentage_error(test_act,test_pred)

    dataframe=dataframe.append(pd.Series([model_description,MAE_Train,MSE_Train,RMSE_Train,MAPE_Train,MAE_Test,MSE_Test,RMSE_Test,MAPE_Test],index=scores.columns ), ignore_index=True)

    return(dataframe)

In [19]:
scores=get_metrics(y_train,lin_model.predict(X_train),y_test,lin_model.predict(X_test),'Lin_model',scores)
scores

<ipython-input-18-b2324f57be52>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe=dataframe.append(pd.Series([model_description,MAE_Train,MSE_Train,RMSE_Train,MAPE_Train,MAE_Test,MSE_Test,RMSE_Test,MAPE_Test],index=scores.columns ), ignore_index=True)


,Model,MAE_Train,MSE_Train,RMSE_Train,MAPE_Train,MAE_Test,MSE_Test,RMSE_Test,MAPE_Test
0,Lin_model,2.084746,4.859845,2.204506,1.255481,2.082794,4.851149,2.202532,1.271057


## Linear Regression Model Using Neural Network

In [20]:
model1 = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(1)
])

In [21]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                270       
                                                                 
 dense_1 (Dense)             (None, 1)                 31        
                                                                 
Total params: 301 (1.18 KB)
Trainable params: 301 (1.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model1.compile(loss="mean_squared_error",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=['mse'])

In [23]:
model1_history = model1.fit(X_train, y_train, epochs=20,
                            validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.7777 - mse: 0.7777 - val_loss: 7.6812 - val_mse: 7.6812
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 0.6378 - mse: 0.6378 - val_loss: 5.4312 - val_mse: 5.4312
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4701 - mse: 0.4701 - val_loss: 6.8593 - val_mse: 6.8593
Epoch 4/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4580 - mse: 0.4580 - val_loss: 1.1926 - val_mse: 1.1926
Epoch 5/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4133 - mse: 0.4133 - val_loss: 0.4488 - val_mse: 0.4488
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4037 - mse: 0.4037 - val_loss: 0.4357 - val_mse: 0.4357
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.3969 - mse: 0.3969 - val_loss: 0.4228 - val_mse: 0.4228
Epoch 8/20
363/363 [==============================] - 1s 3ms/step - loss: 0.

In [24]:
scores=get_metrics(y_train,model1.predict(X_train),
                   y_test,model1.predict(X_test),'LR_NN_Model',scores)
scores

162/162 [==============================] - 0s 752us/step


<ipython-input-18-b2324f57be52>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe=dataframe.append(pd.Series([model_description,MAE_Train,MSE_Train,RMSE_Train,MAPE_Train,MAE_Test,MSE_Test,RMSE_Test,MAPE_Test],index=scores.columns ), ignore_index=True)


,Model,MAE_Train,MSE_Train,RMSE_Train,MAPE_Train,MAE_Test,MSE_Test,RMSE_Test,MAPE_Test
0,Lin_model,2.084746,4.859845,2.204506,1.255481,2.082794,4.851149,2.202532,1.271057
1,LR_NN_Model,0.427289,0.360562,0.600468,0.242864,0.425948,0.361739,0.601447,0.249193


### ANN Model (With Hidden layers)

In [25]:
model2 = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

In [26]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 30)                270       
                                                                 
 dense_3 (Dense)             (None, 30)                930       
                                                                 
 dense_4 (Dense)             (None, 1)                 31        
                                                                 
Total params: 1231 (4.81 KB)
Trainable params: 1231 (4.81 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
model2.compile(loss="mean_squared_error",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=['mse'])

In [28]:
model2_history = model2.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 0.8217 - mse: 0.8217 - val_loss: 4.0043 - val_mse: 4.0043
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4643 - mse: 0.4643 - val_loss: 1.2387 - val_mse: 1.2387
Epoch 3/20
363/363 [==============================] - 1s 1ms/step - loss: 0.4113 - mse: 0.4113 - val_loss: 0.6141 - val_mse: 0.6141
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 0.4062 - mse: 0.4062 - val_loss: 0.3766 - val_mse: 0.3766
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3804 - mse: 0.3804 - val_loss: 0.3663 - val_mse: 0.3663
Epoch 6/20
363/363 [==============================] - 1s 1ms/step - loss: 0.3720 - mse: 0.3720 - val_loss: 0.4670 - val_mse: 0.4670
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 0.3716 - mse: 0.3716 - val_loss: 0.3536 - val_mse: 0.3536
Epoch 8/20
363/363 [==============================] - 1s 1ms/step - loss: 0.

In [29]:
scores=get_metrics(y_train, model2.predict(X_train),y_test,model2.predict(X_test),'ANN_model',scores)
scores

162/162 [==============================] - 0s 854us/step


<ipython-input-18-b2324f57be52>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe=dataframe.append(pd.Series([model_description,MAE_Train,MSE_Train,RMSE_Train,MAPE_Train,MAE_Test,MSE_Test,RMSE_Test,MAPE_Test],index=scores.columns ), ignore_index=True)


,Model,MAE_Train,MSE_Train,RMSE_Train,MAPE_Train,MAE_Test,MSE_Test,RMSE_Test,MAPE_Test
0,Lin_model,2.084746,4.859845,2.204506,1.255481,2.082794,4.851149,2.202532,1.271057
1,LR_NN_Model,0.427289,0.360562,0.600468,0.242864,0.425948,0.361739,0.601447,0.249193
2,ANN_model,0.396516,0.326654,0.571537,0.216258,0.401471,0.334372,0.578249,0.227107


From the results of Three Models, ANN is considered as the best model